<a href="https://colab.research.google.com/github/eshiang21/CS-Project-Zombie-Dash/blob/master/69_CNN_Schirr_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import torch.nn as nn
import torch.nn.functional as F

class CNN_Schirr(nn.Module):
    def __init__(self, input_size, num_electrodes=22, dropout_rate=0.5, filter_width=3, pool_width=3, \
                 c1=50, c2=50, c3=75, c4=75, c5=100, c6=100):
        super(CNN_Schirr, self).__init__()
        self.lr = 0.001
        self.weight_decay = 0.005
        self.name = "CNN_Schirr"

        self.act = F.elu
        self.pool = nn.MaxPool2d((1, pool_width))
        self.drop = nn.Dropout(p=dropout_rate)

        self.c1 = nn.Conv2d(1, c1, (1, filter_width))
        self.bn1 = nn.BatchNorm2d(c1)
        self.c2 = nn.Conv2d(c1, c1, (num_electrodes, 1))
        self.bn2 = nn.BatchNorm2d(c1)
        self.c3 = nn.Conv2d(c1, c2, (1, filter_width))
        self.bn3 = nn.BatchNorm2d(c2)
        self.c4 = nn.Conv2d(c2, c3, (1, filter_width))
        self.bn4 = nn.BatchNorm2d(c3)
        self.c5 = nn.Conv2d(c3, c4, (1, filter_width))
        self.bn5 = nn.BatchNorm2d(c4)

        self.c6 = nn.Conv2d(c4, c5, (1, filter_width))
        self.bn6 = nn.BatchNorm2d(c5)
        

        self.out_size = input_size
        for _ in range(5):
            self.out_size = (self.out_size - filter_width + 1) // pool_width
        self.out_size *= c5
        self.lf = nn.Linear(self.out_size, 5)
    
    def forward(self, x):
        x = self.c1(x)
        x = self.bn1(x)
        x = self.c2(x)
        x = self.act(self.bn2(x))
        x = self.pool(x)
        x = self.drop(x)
        x = self.c3(x)
        x = self.act(self.bn3(x))
        x = self.pool(x)
        x = self.drop(x)
        x = self.c4(x)
        x = self.act(self.bn4(x))
        x = self.pool(x)
        x = self.drop(x)
        x = self.c5(x)
        x = self.act(self.bn5(x))
        x = self.pool(x)

        x = self.drop(x)
        x = self.c6(x)
        x = self.act(self.bn6(x))
        x = self.pool(x)
        

        x = x.view(-1, self.out_size)
        x = self.lf(x)
        return F.log_softmax(x, dim=1)

class RNN(nn.Module):
    def __init__(self, input_size, hidden_dim=100, num_electrodes=22, num_layers=2, dropout_rate=0.8, bi=False, rnn_type="GRU"):
        super(RNN, self).__init__()
        self.name = "RNN"
        self.lr = 0.0005
        self.weight_decay = 0.01

        if rnn_type == "GRU":
            self.rnn = nn.GRU(num_electrodes, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout_rate, bidirectional=bi)
        elif rnn_type == "LSTM":
            self.rnn = nn.LSTM(num_electrodes, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout_rate, bidirectional=bi)
        else:
            print("Invalid rnn type. Must be 'GRU' or 'LSTM'. Got:", rnn_type)
            exit(1)
            
        self.out_size = input_size * hidden_dim
        if bi:
            self.out_size *= 2
        self.drop = nn.Dropout(p=dropout_rate)
        self.fc = nn.Linear(self.out_size, 4)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = x.flatten(start_dim=1)
        x = self.drop(x)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.name = "ConvNet"
        self.act = F.elu
        self.c1 = nn.Conv2d(1, 8, (1, 129), padding=(0, 64))
        self.bn1 = nn.BatchNorm2d(8)
        self.c2 = nn.Conv2d(8, 16, (22, 1))
        self.bn2 = nn.BatchNorm2d(16)
        self.a2 = nn.AvgPool2d((1, 5))
        # self.d2 = nn.Dropout(p=0.3)
        self.c3 = nn.Conv2d(16, 16, (1, 65), groups=16, padding=(0, 32))
        self.c4 = nn.Conv2d(16, 32, 1)
        self.bn4 = nn.BatchNorm2d(32)
        self.a4 = nn.AvgPool2d((1, 5))
        # self.d4 = nn.Dropout(p=0.3)
        self.c5 = nn.Conv2d(32, 32, (1, 33), groups=32, padding=(0, 16))
        self.c6 = nn.Conv2d(32, 64, 1)
        self.bn6 = nn.BatchNorm2d(64)
        self.a6 = nn.AvgPool2d((1, 4))
        self.out_size = 640
        self.lf = nn.Linear(self.out_size, 4)
    
    def forward(self, x):
        # input should have length 1000
        x = self.c1(x)
        x = self.act(self.bn1(x))
        x = self.c2(x)
        x = self.act(self.bn2(x))
        x = self.a2(x)
        # x = self.d2(x)
        x = self.c3(x)
        x = self.c4(x)
        x = self.act(self.bn4(x))
        x = self.a4(x)
        # x = self.d4(x)
        x = self.c5(x)
        x = self.c6(x)
        x = self.act(self.bn6(x))
        x = self.a6(x)
        x = x.view(-1, self.out_size)
        x = self.lf(x)
        return F.log_softmax(x, dim=1)

class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.name = "ResNet"
        self.act = F.elu
        self.c1 = nn.Conv2d(1, 8, (1, 129), padding=(0, 64))
        self.bn1 = nn.BatchNorm2d(8)
        self.c2 = nn.Conv2d(8, 16, (22, 1))
        self.bn2 = nn.BatchNorm2d(16)
        self.a2 = nn.AvgPool2d((1, 2))
        self.c3 = nn.Conv2d(16, 16, (1, 65), groups=16, padding=(0, 32))
        self.c4 = nn.Conv2d(16, 32, 1)
        # self.c4r = nn.Conv2d(16, 32, 1)
        self.bn4 = nn.BatchNorm2d(32)
        self.a4 = nn.AvgPool2d((1, 2))
        self.c5 = nn.Conv2d(32, 32, (1, 33), groups=32, padding=(0, 16))
        self.c6 = nn.Conv2d(32, 64, 1)
        # self.c6r = nn.Conv2d(32, 64, 1)
        self.bn6 = nn.BatchNorm2d(64)
        self.a6 = nn.AvgPool2d((1, 2))
        self.c7 = nn.Conv2d(64, 64, (1, 17), groups=64, padding=(0, 8))
        self.c8 = nn.Conv2d(64, 128, 1)
        # self.c8r = nn.Conv2d(64, 128, 1)
        self.bn8 = nn.BatchNorm2d(128)
        self.a8 = nn.AvgPool2d((1,5))
        self.c9 = nn.Conv2d(128, 128, (1, 9), groups=128, padding=(0, 4))
        self.c10 = nn.Conv2d(128, 256, 1)
        # self.c10r = nn.Conv2d(128, 256, 1)
        self.bn10 = nn.BatchNorm2d(256)
        self.a10 = nn.AvgPool2d((1,5))
        self.out_size = 256 * 4
        self.lf = nn.Linear(self.out_size, 4)
    
    def forward(self, x):
        # input should have length 800
        x = self.c1(x)
        x = self.act(self.bn1(x))
        x = self.c2(x)
        x = self.act(self.bn2(x))
        y = self.a2(x)
        x = self.c3(y)
        x = self.c4(x)
        x = self.act(self.bn4(x))
        y = self.a4(x)
        x = self.c5(y)
        x = self.c6(x)
        x = self.act(self.bn6(x))
        y = self.a6(x)
        x = self.c7(y)
        x = self.c8(x)
        x = self.act(self.bn8(x))
        y = self.a8(x)
        x = self.c9(y)
        x = self.c10(x)
        x = self.act(self.bn10(x))
        x = self.a10(x)
        x = x.view(-1, self.out_size)
        x = self.lf(x)
        return F.log_softmax(x, dim=1)

class ResNetv2(nn.Module):
    def __init__(self):
        self.name = "ResNetv2"
        super(ResNetv2, self).__init__()
        self.act = F.elu

        self.c1 = nn.Conv2d(1, 8, (1, 129), padding=(0, 64))
        self.bn1 = nn.BatchNorm2d(8)

        self.c2 = nn.Conv2d(8, 16, (22, 1))
        self.bn2 = nn.BatchNorm2d(16)
        self.a2 = nn.AvgPool2d((1, 5))

        self.c3r1 = nn.Conv2d(16, 16, (1, 65), groups=16, padding=(0, 32))
        self.c3r2 = nn.Conv2d(16, 16, 1)
        self.bn3r2 = nn.BatchNorm2d(16)
        self.c3r3 = nn.Conv2d(16, 16, (1, 65), groups=16, padding=(0, 32))
        self.c3r4 = nn.Conv2d(16, 16, 1)
        self.bn3r4 = nn.BatchNorm2d(16)

        self.c3 = nn.Conv2d(16, 16, (1, 65), groups=16, padding=(0, 32))
        self.c4 = nn.Conv2d(16, 32, 1)
        self.bn4 = nn.BatchNorm2d(32)
        self.a4 = nn.AvgPool2d((1, 5))

        self.c5r1 = nn.Conv2d(32, 32, (1, 33), groups=32, padding=(0, 16))
        self.c5r2 = nn.Conv2d(32, 32, 1)
        self.bn5r2 = nn.BatchNorm2d(32)
        self.c5r3 = nn.Conv2d(32, 32, (1, 33), groups=32, padding=(0, 16))
        self.c5r4 = nn.Conv2d(32, 32, 1)
        self.bn5r4 = nn.BatchNorm2d(32)

        self.c5 = nn.Conv2d(32, 32, (1, 33), groups=32, padding=(0, 16))
        self.c6 = nn.Conv2d(32, 64, 1)
        self.bn6 = nn.BatchNorm2d(64)
        self.a6 = nn.AvgPool2d((1, 4))

        self.out_size = 64 * 10
        self.lf = nn.Linear(self.out_size, 4)
    
    def forward(self, x):
        # input should have length 1000
        x = self.c1(x)
        x = self.act(self.bn1(x))
        x = self.c2(x)
        x = self.act(self.bn2(x))
        y = self.a2(x)

        x = self.c3r1(y)
        x = self.c3r2(x)
        x = self.act(self.bn3r2(x))
        x = self.c3r3(x)
        x = self.c3r4(x)
        x = self.act(self.bn3r4(x) + y)

        x = self.c3(x)
        x = self.c4(x)
        x = self.act(self.bn4(x))
        y = self.a4(x)

        x = self.c5r1(y)
        x = self.c5r2(x)
        x = self.act(self.bn5r2(x))
        x = self.c5r3(x)
        x = self.c5r4(x)
        x = self.act(self.bn5r4(x) + y)

        x = self.c5(y)
        x = self.c6(x)
        x = self.act(self.bn6(x))
        x = self.a6(x)

        x = x.view(-1, self.out_size)
        x = self.lf(x)
        return F.log_softmax(x, dim=1)

In [0]:
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import os

def GetTrialAccuracy(votes):
    acc = 0
    for key in votes.keys():
        maj_class = np.argmax(votes[key][:4])
        if maj_class == votes[key][4]:
            acc += 1

    acc /= float(len(votes.keys()))
    return 100. * acc

def UpdateClassVotes(votes, preds, targets, trial_ids):
    for pred_class, targ, trial_id in zip(preds, targets, trial_ids):
        pred_class = pred_class.item()
        trial_id = trial_id.item()
        targ = targ.item()
        if trial_id not in votes:
            votes[trial_id] = np.zeros(5)
        votes[trial_id][pred_class] += 1
        votes[trial_id][4] = targ

def UnpackDataLoader(item, device, use_gpu):
    trial_ids = None
    if len(item) == 3:
            data, target, trial_ids = item
    else:
        data, target = item
    if use_gpu:
        data, target = data.to(device), target.to(device)
    return data, target, trial_ids

def TrainNN(net, train_loader, optimizer, epoch, print_every=400, get_trial_acc=False, use_gpu=False, device=None):
    criterion = nn.NLLLoss()
    correct = 0
    train_loss = 0
    net.train()
    votes = {}
    running_loss = 0
    running_acc = 0
    print("Running epoch:", epoch)
    for idx, item in enumerate(train_loader):
        total_idx = idx * train_loader.batch_size
        if total_idx % print_every == 0 and total_idx != 0:
            print("Iter {} - loss: {:.4f}, acc: {:.4f}".format(total_idx, running_loss / print_every, running_acc / print_every))
            running_loss = 0
            running_acc = 0

        data, target, trial_ids = UnpackDataLoader(item, device, use_gpu)

        optimizer.zero_grad()
        net_out = net(data)
        loss = criterion(net_out, target)
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(net.parameters(), 0.5)
        optimizer.step()

        total_loss = loss.item() * len(data)
        train_loss += total_loss
        running_loss += total_loss

        pred = net_out.argmax(dim=1, keepdim=True)
        total_acc = pred.eq(target.view_as(pred)).sum().item()
        running_acc += total_acc
        correct += total_acc
        if get_trial_acc:
            UpdateClassVotes(votes, pred, target, trial_ids)

    train_loss /= len(train_loader.dataset)
    acc = 100. * correct / len(train_loader.dataset)

    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        train_loss, correct, len(train_loader.dataset), acc))
    if get_trial_acc:
        acc = GetTrialAccuracy(votes)
        print('Train Trial Accuracy: {:.2f}%'.format(acc))
    print()

    return train_loss, acc

def TestNN(net, test_loader, epoch, get_trial_acc=False, use_gpu=False, device=None):
    net.eval()
    criterion = nn.NLLLoss(reduction='sum')
    test_loss = 0
    correct = 0
    votes = {}
    with torch.no_grad():
        for item in test_loader:
            data, target, trial_ids = UnpackDataLoader(item, device, use_gpu)

            output = net(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            if get_trial_acc:
                UpdateClassVotes(votes, pred, target, trial_ids)

    test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset), acc))

    if get_trial_acc:
        acc = GetTrialAccuracy(votes)
        print('Test Trial Accuracy: {:.2f}%'.format(acc))
    print()

    return test_loss, acc

def PlotNNResults(epochs, train_loss, train_acc, test_loss, test_acc, plot_type=""):
    fig, axs = plt.subplots(2, sharex=True)
    axs[0].plot(epochs, train_acc, label="Train")
    axs[0].plot(epochs, test_acc, label="Test")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend()
    axs[1].plot(epochs, train_loss, label="Train")
    axs[1].plot(epochs, test_loss, label="Test")
    axs[1].set_ylabel("Loss")
    axs[1].legend()

    fig.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
    plt.grid(False)
    plt.xlabel("Epoch")
    title = "Results" if plot_type == "" else plot_type + " Results"
    fig.suptitle(title)
    plt.show()

def AddDir(filename):
    if not os.path.exists(filename):
        os.makedirs(filename)

def GetTrainingCommand(epochs):
    end_training = False
    while True:
        command = input("Last epoch reached. Enter number of epochs to continue training for (0 if none): ")
        if not command.isdigit():
            print("Invalid option")
        elif int(command) == 0:
            end_training = True
            break
        else:
            epochs += list(range(1 + epochs[-1], 1 + epochs[-1] + int(command)))
            break
    return end_training, epochs

def SaveTrainingResults(net, saved_model, model_epoch, saved_opt, saved_sched, cur_min_loss, train_loss, test_loss, train_acc, test_acc):
    while True:
        key = input("Save model and training history? (y/n): ")
        if key is "y":
            name = input("Enter your name: ")
            extension = input("Enter unique model identifier (to be used as part of filename): ")
            extension = net.name + "_" + extension
            AddDir("training_results/" + name)
            np.savez("training_results/" + name + "/" + extension + ".npz", train_loss=train_loss, train_acc=train_acc, test_loss=test_loss, test_acc=test_acc)
            
            AddDir("saved_models/" + name)
            filename = "saved_models/" + name + "/" + extension + ".pt"
            torch.save({
                'model' : saved_model,
                'epoch' : model_epoch,
                'optimizer' : saved_opt,
                'scheduler' : saved_sched,
                'loss' : cur_min_loss,
            }, filename)

            print("Saved {} model from epoch {} with test loss {}".format(extension, model_epoch, cur_min_loss))
            break
        elif key is "n":
            break
        else:
            print("Invalid option")

def TrainTestModel(net, optimizer, train_loader, test_loader, num_epochs=20, use_gpu=True, get_trial_acc=False, print_every=None):
    device = None
    if use_gpu:
        if torch.cuda.is_available():
            device = torch.device("cuda:0")
            net.to(device)
        else:
            print("GPU not available :(")
            exit(1)

    epochs = list(range(num_epochs))
    last_epoch = epochs[-1]
    if print_every is None:
        print_every = train_loader.batch_size * (len(train_loader.dataset) // (5 * train_loader.batch_size))

    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    cur_min_loss = np.Inf
    cur_best_acc = 0

    break_count = 0

    #change lr annealing#
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.8, patience=10, verbose=True, threshold=0.02)
    try:
        idx = -1
        while True:
            idx += 1
            epoch = epochs[idx]
            train_l, train_a = TrainNN(net, train_loader, optimizer, epoch, print_every=print_every, get_trial_acc=get_trial_acc, use_gpu=use_gpu, device=device)
            test_l, test_a = TestNN(net, test_loader, epoch, get_trial_acc=get_trial_acc, use_gpu=use_gpu, device=device)

            if test_l < cur_min_loss:
                cur_min_loss = test_l
                saved_model = net.state_dict()
                saved_opt = optimizer.state_dict()
                saved_sched = scheduler.state_dict()
                model_epoch = epoch
                break_count = 0
            elif test_l > cur_min_loss:
                break_count += 1

            if test_a > cur_best_acc:
                cur_best_acc = test_a
            
            train_loss.append(train_l)
            train_acc.append(train_a)
            test_loss.append(test_l)
            test_acc.append(test_a)

            if break_count >= 15:
                last_epoch = epoch
                print("Stopping early due to stagnation of test loss")
                break

            scheduler.step(test_l)

            if epoch == epochs[-1]:
                end_training, epochs = GetTrainingCommand(epochs)
                if end_training:
                    break
                        
    except KeyboardInterrupt:
        print("Quit training loop due to keyboard interrupt")
        
    print("\nBest test loss was {}".format(cur_min_loss))
    print("Best test accuracy was {}\n".format(cur_best_acc))
    if not use_gpu:
        PlotNNResults(range(last_epoch+1), train_loss, train_acc, test_loss, test_acc)

    SaveTrainingResults(net, saved_model, model_epoch, saved_opt, saved_sched, cur_min_loss, train_loss, test_loss, train_acc, test_acc)

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

def AddNoiseData(data, labels, trials, noise_factor):
    data_set = [data] 
    label_set = [labels]
    trials *= (noise_factor + 1)
    for _ in range(noise_factor):
        noise = np.random.normal(0, 5.0, data.shape)
        data_set.append(data + noise)
        label_set.append(labels)

    data_set = np.vstack(data_set)
    label_set = np.hstack(label_set)

    return data_set, label_set, trials

def ApplySlidingWindow(data_set, label_set, trials, bin_size, start_width, slide_factor, random_choice=False):
    data_res, label_res, trial_res = [], [], []
    for i in range(len(data_set)):
        if random_choice:
            starts = np.random.choice(start_width, slide_factor, replace=False)
        else:
            starts = range(0, start_width+1, start_width//(slide_factor-1))
        for start in starts:
            data_res.append(data_set[i, :, start:start+bin_size])
            label_res.append(label_set[i])
            trial_res.append(trials[i])

    return np.array(data_res), np.array(label_res), trial_res

def SubsampleData(data, labels, trials, subsample_factor):
    ret_data, ret_labels = [], []
    trials *= subsample_factor
    for i in range(subsample_factor):
        ret_data.append(data[:, :, i::subsample_factor])
        ret_labels.append(labels)
    return np.vstack(ret_data), np.hstack(ret_labels), trials

def AugmentData(data, labels, noise_factor=0, slide_factor=1, subsample_factor=1, bin_size=800, augment=False, train_data=True, model_type="conv"):
    trials = list(range(len(data)))
    max_width = 1000 // subsample_factor
    start_width = max_width - bin_size
    if start_width < 0:
        print("Invalid bin size and subsample configuration. Bin size must be <= 1000//subsample_factor. Given bin_size={}, subsample_factor={}.".format(bin_size, subsample_factor))
        exit(1)

    if augment:
        if subsample_factor > 1:
            data, labels, trials = SubsampleData(data, labels, trials, subsample_factor)

        if train_data and noise_factor > 0:
            data, labels, trials = AddNoiseData(data, labels, trials, noise_factor)

        if slide_factor > 1:
            data, labels, trials = ApplySlidingWindow(data, labels, trials, bin_size, start_width, slide_factor)
        else:
            data = data[:, :, start_width//2:start_width//2+bin_size]

    else:
        data = data[:, :, start_width//2:start_width//2+bin_size]

    if model_type == "conv":
        data = data[:, np.newaxis]
    elif model_type == "rec":
        data = data.swapaxes(1, 2)

    return data, labels, np.array(trials)

def GetData(noise_factor=0, slide_factor=1, subsample_factor=1, bin_size=1000, augment_train=False, augment_test=False, model_type="conv"):
    X_test = np.load("/content/gdrive/My Drive/Colab Notebooks/project/X_test.npy")
    y_test = np.load("/content/gdrive/My Drive/Colab Notebooks/project/y_test.npy")
    person_train_valid = np.load("/content/gdrive/My Drive/Colab Notebooks/project/person_train_valid.npy")
    X_train_valid = np.load("/content/gdrive/My Drive/Colab Notebooks/project/X_train_valid.npy")
    y_train_valid = np.load("/content/gdrive/My Drive/Colab Notebooks/project/y_train_valid.npy")
    person_test = np.load("/content/gdrive/My Drive/Colab Notebooks/project/person_test.npy")

    class_map = {769:0, 770:1, 771:2, 772:3}
    y_train_valid = np.array([class_map[x] for x in y_train_valid])
    y_test = np.array([class_map[x] for x in y_test])

    X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.2, stratify=y_train_valid, random_state=0)
    X_train, y_train, train_trials = AugmentData(X_train, y_train, noise_factor, slide_factor, subsample_factor, bin_size, augment=augment_train, train_data=True, model_type=model_type)
    X_valid, y_valid, valid_trials = AugmentData(X_valid, y_valid, noise_factor, slide_factor, subsample_factor, bin_size, augment=augment_test, train_data=False, model_type=model_type)
    X_test, y_test, test_trials = AugmentData(X_test, y_test, noise_factor, slide_factor, subsample_factor, bin_size, augment=augment_test, train_data=False, model_type=model_type)
    
    print("Data shape:", X_train.shape)
    print("Target shape:", y_train.shape)
    print("Trial shape:", train_trials.shape)
    return X_train, y_train, X_valid, y_valid, X_test, y_test, train_trials, valid_trials, test_trials

In [5]:
!ls "/content/gdrive/My Drive/Colab Notebooks/project/"

EEG_loading.ipynb  person_train_valid.npy  X_train_valid.npy  y_train_valid.npy
person_test.npy    X_test.npy		   y_test.npy


In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import torch
import torch.utils.data as utils
from torch import optim
import argparse

def ClassifySVM(X_train_valid, y_train_valid):
    X_train_valid = X_train_valid.reshape((len(X_train_valid), -1))
    print("Data shape:", X_train_valid.shape)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, stratify=y_train_valid, random_state=0)

    svm = SVC()
    svm.fit(X_train, y_train)
    train_acc = np.mean(svm.predict(X_train) == y_train)
    scores = svm.predict(X_valid)
    val_acc = np.mean(scores == y_valid)

    print("Train acc:", train_acc)
    print("Val acc:", val_acc)

def GetTensorLoader(data, labels, trials=None, batch_size=200):
    data_tensor = torch.Tensor(data)
    label_tensor = torch.Tensor(labels).long()
    if trials is None:
        dataset = utils.TensorDataset(data_tensor, label_tensor)
    else:
        trial_tensor = torch.Tensor(trials).long()
        dataset = utils.TensorDataset(data_tensor, label_tensor, trial_tensor)
    loader = utils.DataLoader(dataset, batch_size=batch_size)
    return loader

def main(bin_size, num_epochs, filter_width, augment_train, augment_test, slide_factor, noise_factor, subsample_factor, model_type):
    if model_type != "conv" and model_type != "rec" and model_type != "other":
        print("Model type must be either 'conv', 'rec', or 'other'. Got:", model_type)
        exit(1)

    use_gpu = True

    X_train, y_train, X_valid, y_valid, X_test, y_test, train_trials, valid_trials, test_trials = \
        GetData(noise_factor, slide_factor, subsample_factor, bin_size, augment_train, augment_test, model_type)

    #TODO: reshape your data here as needed

    train_loader = GetTensorLoader(X_train, y_train, train_trials, batch_size=200)
    valid_loader = GetTensorLoader(X_valid, y_valid, valid_trials, batch_size=200)
    
    #TODO: specify your network here

    net = CNN_Schirr(input_size = bin_size, filter_width = filter_width, dropout_rate = 0.4)
    #net = RNN(bin_size)
    optimizer = optim.Adam(net.parameters(), net.lr, weight_decay=net.weight_decay) # TODO: determine best optimizer/params
    print("Number of parameters for {}: {}\n".format(net.name, sum(p.numel() for p in net.parameters() if p.requires_grad)))
    TrainTestModel(net, optimizer, train_loader, valid_loader, num_epochs=num_epochs, use_gpu=use_gpu, get_trial_acc=augment_test)

main(bin_size = 1000, num_epochs = 200, augment_train = True, augment_test = True , \
     slide_factor = 1, noise_factor = 0, subsample_factor = 1, model_type = 'conv', filter_width = 7)

Data shape: (1692, 1, 22, 1000)
Target shape: (1692,)
Trial shape: (1692,)
Number of parameters for CNN_Schirr: 192680

Running epoch: 0
Iter 200 - loss: 2.0151, acc: 0.1000
Iter 400 - loss: 1.8283, acc: 0.1450
Iter 600 - loss: 1.7143, acc: 0.1800
Iter 800 - loss: 1.7499, acc: 0.1600
Iter 1000 - loss: 1.6280, acc: 0.2700
Iter 1200 - loss: 1.6240, acc: 0.2600
Iter 1400 - loss: 1.5400, acc: 0.2900
Iter 1600 - loss: 1.5696, acc: 0.2350
Train set: Average loss: 1.6991, Accuracy: 354/1692 (20.92%)
Train Trial Accuracy: 26.71%

Test set: Average loss: 1.6517, Accuracy: 106/423 (25.06%)
Test Trial Accuracy: 25.06%

Running epoch: 1
Iter 200 - loss: 1.4733, acc: 0.3400
Iter 400 - loss: 1.5055, acc: 0.3000
Iter 600 - loss: 1.4799, acc: 0.2450
Iter 800 - loss: 1.4141, acc: 0.3050
Iter 1000 - loss: 1.4481, acc: 0.3050
Iter 1200 - loss: 1.4487, acc: 0.3400
Iter 1400 - loss: 1.4652, acc: 0.2800
Iter 1600 - loss: 1.4123, acc: 0.2950
Train set: Average loss: 1.4560, Accuracy: 511/1692 (30.20%)
Train 